In [1]:
!pip install transformers
!pip install sentencepiece
!pip install tqdm
from transformers import MarianMTModel, MarianTokenizer

In [2]:
# Get the name of the first model
first_model_name = 'Helsinki-NLP/opus-mt-en-fr'

# Get the tokenizer
first_model_tkn = MarianTokenizer.from_pretrained(first_model_name)

# Load the pretrained model based on the name
first_model = MarianMTModel.from_pretrained(first_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [3]:
# Get the name of the second model
second_model_name = 'Helsinki-NLP/opus-mt-fr-en'

# Get the tokenizer
second_model_tkn = MarianTokenizer.from_pretrained(second_model_name)

# Load the pretrained model based on the name
second_model = MarianMTModel.from_pretrained(second_model_name)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
original_texts = ["This article aims to perform the back translation for text data augmentation",
          "It is the 25th article by Zoumana on Medium. He loves to give back to the community",
          "The first model translates from English to French, which is a temporary process",
          "The second model finally translates back all the temporary french text into English",
          "define airline x3-1 , names of airports and also show me city served both by x16-2 and x18-3 x19-4 x20-5"]

original_texts

['This article aims to perform the back translation for text data augmentation',
 'It is the 25th article by Zoumana on Medium. He loves to give back to the community',
 'The first model translates from English to French, which is a temporary process',
 'The second model finally translates back all the temporary french text into English',
 'define airline x3-1 , names of airports and also show me city served both by x16-2 and x18-3 x19-4 x20-5']

In [4]:
def format_batch_texts(language_code, batch_texts):

    formated_bach = [">>{}<< {}".format(language_code, text) for text in batch_texts]

    return formated_bach

In [5]:
def perform_translation(batch_texts, model, tokenizer, language="fr"):
    # Prepare the text data into appropriate format for the model
    formated_batch_texts = format_batch_texts(language, batch_texts)

    # Generate translation using model
    translated = model.generate(**tokenizer(formated_batch_texts, return_tensors="pt", padding=True))

    # Convert the generated tokens indices back into text
    translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

    return translated_texts

In [6]:
def perform_back_translation(batch_texts, original_language="en", temporary_language="fr"):

  # Translate from Original to Temporary Language
  tmp_translated_batch = perform_translation(batch_texts, first_model, first_model_tkn, temporary_language)

  # Translate Back to English
  back_translated_batch = perform_translation(tmp_translated_batch, second_model, second_model_tkn, original_language)

  # Return The Final Result
  return back_translated_batch

In [7]:
def combine_texts(original_texts, back_translated_batch):

  return set(original_texts + back_translated_batch)

In [ ]:
back_translated_batch = perform_back_translation(original_texts)
back_translated_batch

['This article aims to perform back translation for text data increase',
 "This is Zoumana's 25th article on Medium. He likes to give back to the community.",
 'The first model translates from English to French, which is a temporary process',
 'The second model finally translates all the temporary French text into English',
 'Definition of the airline x3-1, airport names and also show me the city served by both x16-2 and x18-3 x19-4 x20-5']

In [8]:

def mark_entities(sentence, labels):
    marked_sentence = []
    entity_map = {}

    for i, (word, label) in enumerate(zip(sentence, labels)):
        if label != 'O':
            entity_key = f'x{i+1}-{len(entity_map) + 1}'
            marked_sentence.append(entity_key)
            entity_map[entity_key] = (word, label)
        else:
            marked_sentence.append(word)

    return marked_sentence, entity_map

def unmark_entities(sentence, entity_map):
    unmarked_sentence = []
    labels=[]
    # print(entity_map)
    # print(sentence)
    for word in sentence:
        if word.startswith('x'):
            entity_key = word
            if entity_key[-1]==',' or entity_key[-1]=='.'or entity_key[-1]=='?':
                entity_key = entity_key[:-1]

            original_word, original_label = entity_map[entity_key]
            unmarked_sentence.append(original_word)
            labels.append(original_label)
        else:
            labels.append('O')
            unmarked_sentence.append(word)

    return unmarked_sentence, labels

def translate_with_entity_mask(sentence, labels):

    # Mark entities in the original sentence
    marked_sentence, entity_map = mark_entities(sentence, labels)
    marked_sentence_str = ' '.join(marked_sentence)


    backtranslated_sentence = perform_back_translation([marked_sentence_str], "en", "fr")

    backtranslated_sentence_list = backtranslated_sentence[0].split()

    unmarked_backtranslated_sentence, new_labels = unmark_entities(backtranslated_sentence_list, entity_map)

    return unmarked_backtranslated_sentence, new_labels



def augment_data_sense_for_sense(sentence, labels):
    return translate_with_entity_mask(sentence, labels)


# Example usage
sentence = ['define', 'airline', 'ua', ',', 'names', 'of', 'airports', 'and', 'also', 'show', 'me', 'city', 'served', 'both', 'by', 'nationair', 'and', 'canadian', 'airlines', 'international']
labels = ['O', 'O', 'B-airline_code', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-airline_name', 'O', 'B-airline_name', 'I-airline_name', 'I-airline_name']


augmented_sentences, augmented_labels = augment_data_sense_for_sense(sentence, labels)

# Print augmented data for demonstration
print("Original Sentence:", sentence)
print("Original Labels:", labels)
print("Augmented Sentences:", augmented_sentences)
print("Augmented Labels:", augmented_labels)

Original Sentence: ['define', 'airline', 'ua', ',', 'names', 'of', 'airports', 'and', 'also', 'show', 'me', 'city', 'served', 'both', 'by', 'nationair', 'and', 'canadian', 'airlines', 'international']
Original Labels: ['O', 'O', 'B-airline_code', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-airline_name', 'O', 'B-airline_name', 'I-airline_name', 'I-airline_name']
Augmented Sentences: ['Definition', 'of', 'the', 'airline', 'ua', 'airport', 'names', 'and', 'also', 'show', 'me', 'the', 'city', 'served', 'by', 'both', 'nationair', 'and', 'canadian', 'airlines', 'international']
Augmented Labels: ['O', 'O', 'O', 'O', 'B-airline_code', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-airline_name', 'O', 'B-airline_name', 'I-airline_name', 'I-airline_name']


In [9]:
!git clone https://github.com/VinAIResearch/MISCA.git

Cloning into 'MISCA'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 119 (delta 59), reused 69 (delta 24), pack-reused 0
Receiving objects: 100% (119/119), 2.64 MiB | 6.08 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [10]:
def read_file( input_file, quotechar=None):
        """ Read data file of given path.
        :param file_path: path of data file.
        :return: list of sentence, list of slot and list of intent.
        """

        texts, slots, intents = [], [], []
        text, slot = [], []

        with open(input_file, 'r', encoding="utf8") as fr:
            for line in fr.readlines():
                items = line.strip().split()

                if len(items) == 1:
                    texts.append(text)
                    slots.append(slot)
                    if "/" not in items[0]:
                        intents.append(items)
                    else:
                        new = items[0].split("/")
                        intents.append([new[1]])

                    # clear buffer lists.
                    text, slot = [], []

                elif len(items) == 2:
                    text.append(items[0].strip())
                    slot.append(items[1].strip())

        return texts, slots, intents

In [13]:
texts,slots,intents = read_file("MISCA/data/mixatis/train.txt")

In [ ]:
print(texts[0])
print(slots[0])
print(intents[0])
print(len(texts))
print(len(slots))
print(len(intents))

['define', 'airline', 'ua', ',', 'names', 'of', 'airports', 'and', 'also', 'show', 'me', 'city', 'served', 'both', 'by', 'nationair', 'and', 'canadian', 'airlines', 'international']
['O', 'O', 'B-airline_code', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-airline_name', 'O', 'B-airline_name', 'I-airline_name', 'I-airline_name']
['atis_abbreviation#atis_airport#atis_city']
13162
13162
13162


In [ ]:
import tqdm
from tqdm import tqdm
def write_file(output_file, texts, slots, intents):
    """ Write data file of given path.
    :param file_path: path of data file.
    :return: list of sentence, list of slot and list of intent.
    """
    with open(output_file, 'w', encoding='utf8') as fo:
        for text, slot, intent in (zip(texts, slots, intents)):
            for t, s in zip(text, slot):
                fo.write(f"{t} {s}\n")
            fo.write(f"{intent[0]}\n\n")

In [ ]:
texts,slots,intents = read_file("MISCA/data/mixatis/train.txt")
new_texts,new_slots,new_intents = [],[],[]
texts = texts[9000:13000]
slots = slots[9000:13000]
intents = intents[9000:13000]

for text, slot, intent in tqdm(zip(texts, slots, intents)):
    try:
        augmented_sentences, augmented_labels = augment_data_sense_for_sense(text, slot)
        new_texts.append(augmented_sentences)
        new_slots.append(augmented_labels)
        new_intents.append(intent)
    except:
      pass

write_file("train_augmented(9000-13000).txt", new_texts, new_slots, new_intents)


4000it [1:44:29,  1.57s/it]


In [ ]:
new_texts,new_slots,new_intents = read_file("train_augmented(0-2000).txt")
texts,slots,intents = read_file("MISCA/data/mixatis/train.txt")

print(len(texts))
print(len(slots))
print(len(intents))
print(len(new_texts))
print(len(new_slots))
print(len(new_intents))

13162
13162
13162
2000
2000
2000


In [ ]:
print(new_texts[-1])
print(new_slots[-1])
print(new_intents[-1])

['What', 'ord', 'means']
['O', 'B-airport_code', 'O']
['atis_abbreviation']


In [ ]:
write_file("train_augmented(2000-5551).txt", new_texts, new_slots, new_intents)
